# Part 1: Analysis

## Analyze and extract any interesting insights you can derive from the data set attached (each row represents the assignment of a job in our research queue, including some data about the analyst who received the assignment and the current state of the research queue). What can you infer? What do you think this means for us from a business perspective?

In [61]:
# Import necessary libraries

import sqlite3
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [103]:
# Load data
df = pd.read_csv('Assignment_Log.csv')
df['id'] = df.index

# Strip whitespace from headers
df.columns = df.columns.str.strip()

#Replace spaces from headers with _
df.columns = df.columns.str.replace(" ","_")

#Remove paraentheses from headers
df.columns = df.columns.str.replace("(", "")
df.columns = df.columns.str.replace(")", "")

# Fix datetime columns to correct format
for i in range(len(df.Event_occurred_at)):
    if len(df.Event_occurred_at[i]) == 18:
        df['Event_occurred_at'][i] = df['Event_occurred_at'][i][:11] + "0" + df['Event_occurred_at'][i][11:]

for j in range(len(df.Request_created_at)):
    if len(df.Request_created_at[j]) == 18:
        df['Request_created_at'][j] = df['Request_created_at'][j][:11] + "0" + df['Request_created_at'][j][11:]


# Convert / in datetime columns to -
df.Event_occurred_at = df.Event_occurred_at.str.replace("/", "-")
df.Request_created_at = df.Request_created_at.str.replace("/", "-")

# Convert datetime columns to datetime objects
for k in range(len(df.Event_occurred_at)):
    df['Event_occurred_at'] = pd.to_datetime(df['Event_occurred_at'])
    
for l in range(len(df.Request_created_at)):
    df['Request_created_at'] = pd.to_datetime(df['Request_created_at'])

# Connect to databse
conn = sqlite3.connect('database.db')
c = conn.cursor()

# Insert data into database
df.to_sql("Assignments", conn)

/Users/erikadauria/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/erikadauria/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [104]:
df.head()

,Event_occurred_at,Analyst,Quality_score_sourcing,Quality_score_writing,Action,Request,Request_created_at,Job,Wait_time_min,Waiting_for,Analysts_available,Analysts_occupied,Total_jobs_available,Review_jobs_available,Vetting_jobs_available,Planning_jobs_available,Editing_jobs_available,Sourcing_jobs_available,Writing_jobs_available,id
0,2017-06-22 19:59:06,9fcbc63ff4c8bea5cea4efad782c87cf,5.00,5.00,Accepted Job,594bec5c95e2ce005840c23a,2017-06-22 12:12:12,review,1,review,0,13,14,4,6,2,0,1,1,0
1,2017-06-22 19:59:02,9fcbc63ff4c8bea5cea4efad782c87cf,5.00,5.00,Assigned Job,594bec5c95e2ce005840c23a,2017-06-22 12:12:12,review,1,review,1,13,15,5,6,2,0,1,1,1
2,2017-06-22 19:51:30,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Declined Job,594bec83fd2cf400280aa965,2017-06-22 12:12:51,writing,9,"sourcing, writing",1,12,12,5,5,1,0,0,1,2
3,2017-06-22 19:51:01,0e9802516f8a79dd0d45211dd4ee74af,4.50,4.50,Accepted Job,594c1f5cd7e68f0028c9062c,2017-06-22 15:49:48,sourcing,1,"sourcing, writing",1,11,12,5,5,1,0,0,1,3
4,2017-06-22 19:50:58,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Assigned Job,594bec83fd2cf400280aa965,2017-06-22 12:12:51,writing,8,"sourcing, writing",2,11,14,5,5,2,0,1,1,4


In [105]:
# Check data types in Assignments table

c. execute('PRAGMA table_info(Assignments);')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Event_occurred_at,TIMESTAMP,0,None,0
2,2,Analyst,TEXT,0,None,0
3,3,Quality_score_sourcing,REAL,0,None,0
4,4,Quality_score_writing,REAL,0,None,0
5,5,Action,TEXT,0,None,0
6,6,Request,TEXT,0,None,0
7,7,Request_created_at,TIMESTAMP,0,None,0
8,8,Job,TEXT,0,None,0
9,9,Wait_time_min,INTEGER,0,None,0


In [101]:
#c.execute('DROP TABLE Assignments;')

In [106]:
# Query data from Assignments table

c.execute('''SELECT * FROM Assignments;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,index,Event_occurred_at,Analyst,Quality_score_sourcing,Quality_score_writing,Action,Request,Request_created_at,Job,Wait_time_min,Waiting_for,Analysts_available,Analysts_occupied,Total_jobs_available,Review_jobs_available,Vetting_jobs_available,Planning_jobs_available,Editing_jobs_available,Sourcing_jobs_available,Writing_jobs_available,id
0,0,2017-06-22 19:59:06,9fcbc63ff4c8bea5cea4efad782c87cf,5.00,5.00,Accepted Job,594bec5c95e2ce005840c23a,2017-06-22 12:12:12,review,1,review,0,13,14,4,6,2,0,1,1,0
1,1,2017-06-22 19:59:02,9fcbc63ff4c8bea5cea4efad782c87cf,5.00,5.00,Assigned Job,594bec5c95e2ce005840c23a,2017-06-22 12:12:12,review,1,review,1,13,15,5,6,2,0,1,1,1
2,2,2017-06-22 19:51:30,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Declined Job,594bec83fd2cf400280aa965,2017-06-22 12:12:51,writing,9,"sourcing, writing",1,12,12,5,5,1,0,0,1,2
3,3,2017-06-22 19:51:01,0e9802516f8a79dd0d45211dd4ee74af,4.50,4.50,Accepted Job,594c1f5cd7e68f0028c9062c,2017-06-22 15:49:48,sourcing,1,"sourcing, writing",1,11,12,5,5,1,0,0,1,3
4,4,2017-06-22 19:50:58,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Assigned Job,594bec83fd2cf400280aa965,2017-06-22 12:12:51,writing,8,"sourcing, writing",2,11,14,5,5,2,0,1,1,4
5,5,2017-06-22 19:50:58,0e9802516f8a79dd0d45211dd4ee74af,4.50,4.50,Assigned Job,594c1f5cd7e68f0028c9062c,2017-06-22 15:49:48,sourcing,1,"sourcing, writing",2,11,14,5,5,2,0,1,1,5
6,6,2017-06-22 19:50:58,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Declined Job,594bec83fd2cf400280aa965,2017-06-22 12:12:51,writing,8,"sourcing, writing",2,11,14,5,5,2,0,1,1,6
7,7,2017-06-22 19:50:23,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Assigned Job,594bec83fd2cf400280aa965,2017-06-22 12:12:51,writing,8,"sourcing, writing",1,11,14,5,5,2,0,1,1,7
8,8,2017-06-22 19:47:03,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Accepted Job,594c1e983b593b00281250ba,2017-06-22 15:46:32,sourcing,4,"sourcing, writing",1,11,11,5,5,1,0,0,0,8
9,9,2017-06-22 19:47:03,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Accepted Job,594c1e983b593b00281250ba,2017-06-22 15:46:32,sourcing,4,"sourcing, writing",1,11,11,5,5,1,0,0,0,9


## Duplicates

In [107]:
# Query the duplicate rows in this dataset

c.execute('''SELECT Event_occurred_at, Analyst, Quality_score_sourcing, Quality_score_writing, Action, Request, Request_created_at, Job, Wait_time_min, Waiting_for, Analysts_available, Analysts_occupied, Total_jobs_available, Review_jobs_available, Vetting_jobs_available, Planning_jobs_available, Editing_jobs_available, Sourcing_jobs_available, Writing_jobs_available, COUNT(*)
            FROM Assignments
            GROUP BY Event_occurred_at, Analyst, Quality_score_sourcing, Quality_score_writing, Action, Request, Request_created_at, Job, Wait_time_min, Waiting_for, Analysts_available, Analysts_occupied, Total_jobs_available, Review_jobs_available, Vetting_jobs_available, Planning_jobs_available, Editing_jobs_available, Sourcing_jobs_available, Writing_jobs_available
            HAVING COUNT(*) > 1;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data.head(15)

,Event_occurred_at,Analyst,Quality_score_sourcing,Quality_score_writing,Action,Request,Request_created_at,Job,Wait_time_min,Waiting_for,Analysts_available,Analysts_occupied,Total_jobs_available,Review_jobs_available,Vetting_jobs_available,Planning_jobs_available,Editing_jobs_available,Sourcing_jobs_available,Writing_jobs_available,COUNT(*)
0,2017-06-22 18:26:42,e2333b2dc03032f12c8526e45243f0c1,0.00,0.00,Accepted Job,594c0b883b593b002812506e,2017-06-22 14:25:12,sourcing,22,sourcing,2,14,9,2,7,0,0,0,0,2
1,2017-06-22 19:47:03,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Accepted Job,594c1e983b593b00281250ba,2017-06-22 15:46:32,sourcing,4,"sourcing, writing",1,11,11,5,5,1,0,0,0,2


In [112]:
# Remove the duplicate rows in this dataset

c.execute('''DELETE FROM Assignments
             WHERE id NOT IN (SELECT MAX(id)
                FROM Assignments
                GROUP BY Event_occurred_at, Analyst, Quality_score_sourcing, Quality_score_writing, Action, Request, Request_created_at, Job, Wait_time_min, Waiting_for, Analysts_available, Analysts_occupied, Total_jobs_available, Review_jobs_available, Vetting_jobs_available, Planning_jobs_available, Editing_jobs_available, Sourcing_jobs_available, Writing_jobs_available);''')

In [113]:
# Query the new count of the Assignments datatable

c.execute('''SELECT COUNT(*) FROM Assignments;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,COUNT(*)
0,789


## Time

In [6]:
# Query the minimum and maximum values for Event_occurred_at

c.execute('''SELECT MIN(Event_occurred_at) AS START_TIME, MAX(Event_occurred_at) AS END_TIME
            FROM Assignments;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data.head(15)

,START_TIME,END_TIME
0,2017-06-21 20:15:42,2017-06-22 19:59:06


The events logged in this data table are between __06/21/2017 20:15:42__ to __06/22/2017 19:59:06__, about a days worth of events.

In [9]:
# Query the minimum and maximum values for Request_created_at

c.execute('''SELECT MIN(Request_created_at) AS START_TIME, MAX(Request_created_at) AS END_TIME
            FROM Assignments;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data.head(15)

,START_TIME,END_TIME
0,2017-06-19 13:43:51,2017-06-22 18:16:11


The events logged in this data table are for requests created between __06/19/2017 13:43:51__ to __06/22/2017 18:16:11__, about 3 days worth of requests.

In [21]:
# Query the time it takes for activity to start on a request

c.execute('''SELECT Request, ((strftime('%s', MIN(Event_occurred_at)) - strftime('%s', Request_created_at))/60) AS Time_for_activity_to_begin_mins
            FROM Assignments
            WHERE Action LIKE '%Accepted Job%'
            GROUP BY Request
            HAVING Request_created_at >= '2017-06-21 20:15:42'
            ORDER BY Request_created_at;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data.head(10)

,Request,Time_for_activity_to_begin_mins
0,594b1abfa6992500289d6040,240
1,594b20ea3f82d10028f03c54,12
2,594b20eb3f82d10028f03c5c,29
3,594b20ec3f82d10028f03c64,33
4,594b20ed3f82d10028f03c6c,48
5,594b20ee3f82d10028f03c74,98
6,594b20ef3f82d10028f03c7c,107
7,594b31f09d87570035cccafd,153
8,594b33d6fee3760035a1a540,33
9,594b52f717d10a00289676dd,157


In [19]:
# Query the time it takes for activity to start on a request

c.execute('''SELECT Request, Request_created_at, MIN(Event_occurred_at), ((strftime('%s', MIN(Event_occurred_at)) - strftime('%s', Request_created_at))/60) AS Time_for_activity_to_begin_mins
            FROM Assignments
            WHERE Action LIKE '%Accepted Job%'
            GROUP BY Request
            HAVING Request_created_at >= '2017-06-21 20:15:42'
            ORDER BY Request_created_at;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,Request,Request_created_at,MIN(Event_occurred_at),Time_for_activity_to_begin_mins
0,594b1abfa6992500289d6040,2017-06-21 21:17:51,2017-06-22 01:17:52,240
1,594b20ea3f82d10028f03c54,2017-06-21 21:44:10,2017-06-21 21:56:48,12
2,594b20eb3f82d10028f03c5c,2017-06-21 21:44:11,2017-06-21 22:14:00,29
3,594b20ec3f82d10028f03c64,2017-06-21 21:44:12,2017-06-21 22:17:14,33
4,594b20ed3f82d10028f03c6c,2017-06-21 21:44:13,2017-06-21 22:33:07,48
5,594b20ee3f82d10028f03c74,2017-06-21 21:44:14,2017-06-21 23:23:02,98
6,594b20ef3f82d10028f03c7c,2017-06-21 21:44:15,2017-06-21 23:31:16,107
7,594b31f09d87570035cccafd,2017-06-21 22:56:48,2017-06-22 01:30:23,153
8,594b33d6fee3760035a1a540,2017-06-21 23:04:54,2017-06-21 23:38:17,33
9,594b52f717d10a00289676dd,2017-06-22 01:17:43,2017-06-22 03:55:15,157


In [14]:
# Query the average time it takes for activity to start on a request

c.execute('''SELECT AVG(Time_for_activity_to_begin) AS Average_time_for_activity_to_begin_mins
            FROM (SELECT Request, ((strftime('%s', MIN(Event_occurred_at)) - strftime('%s', Request_created_at))/60) AS Time_for_activity_to_begin
            FROM Assignments
            WHERE Action LIKE '%Accepted Job%'
            GROUP BY Request
            HAVING Request_created_at >= '2017-06-21 20:15:42'
            ORDER BY Request_created_at);''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,Average_time_for_activity_to_begin_mins
0,69.372093


The average time for activity to begin on a request is about 69 minutes or a little more than an hour.

In [23]:
# Query the average time it takes for activity to start on a request without the 393 outlier

c.execute('''SELECT AVG(Time_for_activity_to_begin) AS Average_time_for_activity_to_begin_mins
            FROM (SELECT Request, ((strftime('%s', MIN(Event_occurred_at)) - strftime('%s', Request_created_at))/60) AS Time_for_activity_to_begin
            FROM Assignments
            WHERE Action LIKE '%Accepted Job%' 
            GROUP BY Request
            HAVING Request_created_at >= '2017-06-21 20:15:42' AND Time_for_activity_to_begin < 393
            ORDER BY Request_created_at);''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,Average_time_for_activity_to_begin_mins
0,61.666667


The average time for activity to begin on a request without the 393 outlier is about 61.6 minutes or a little more than an hour.

In [39]:
c.execute('''SELECT COUNT(*)
            FROM Assignments
            WHERE Action LIKE  '%Accepted Job';''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data


,COUNT(*)
0,303


In [37]:
c.execute('''SELECT COUNT(*)
            FROM Assignments
            WHERE Action LIKE  '%Declined Job';''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data


,COUNT(*)
0,95


In [42]:
c.execute('''SELECT COUNT(*)
            FROM Assignments
            WHERE Action LIKE  '%Assigned Job';''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data


,COUNT(*)
0,393


In [45]:
c.execute('''SELECT *
            FROM Assignments
            WHERE Action LIKE  '%Assigned Job';''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data


,index,Event_occurred_at,Analyst,Quality_score_sourcing,Quality_score_writing,Action,Request,Request_created_at,Job,Wait_time_min,Waiting_for,Analysts_available,Analysts_occupied,Total_jobs_available,Review_jobs_available,Vetting_jobs_available,Planning_jobs_available,Editing_jobs_available,Sourcing_jobs_available,Writing_jobs_available
0,1,2017-06-22 19:59:02,9fcbc63ff4c8bea5cea4efad782c87cf,5.00,5.00,Assigned Job,594bec5c95e2ce005840c23a,2017-06-22 12:12:12,review,1,review,1,13,15,5,6,2,0,1,1
1,4,2017-06-22 19:50:58,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Assigned Job,594bec83fd2cf400280aa965,2017-06-22 12:12:51,writing,8,"sourcing, writing",2,11,14,5,5,2,0,1,1
2,5,2017-06-22 19:50:58,0e9802516f8a79dd0d45211dd4ee74af,4.50,4.50,Assigned Job,594c1f5cd7e68f0028c9062c,2017-06-22 15:49:48,sourcing,1,"sourcing, writing",2,11,14,5,5,2,0,1,1
3,7,2017-06-22 19:50:23,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Assigned Job,594bec83fd2cf400280aa965,2017-06-22 12:12:51,writing,8,"sourcing, writing",1,11,14,5,5,2,0,1,1
4,10,2017-06-22 19:46:56,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Assigned Job,594c1e983b593b00281250ba,2017-06-22 15:46:32,sourcing,4,"sourcing, writing",2,12,13,5,5,2,0,1,0
5,12,2017-06-22 19:44:01,a09c8906073b4c0b75e3100b857b982a,5.00,5.00,Assigned Job,594bec5b95e2ce005840c232,2017-06-22 12:12:11,review,1,"review, planning, editing, sourcing, writing",2,10,12,6,5,1,0,0,0
6,14,2017-06-22 19:40:37,0e9802516f8a79dd0d45211dd4ee74af,4.50,4.50,Assigned Job,594bec83fd2cf400280aa965,2017-06-22 12:12:51,writing,1,"sourcing, writing",3,9,13,6,5,1,0,0,1
7,16,2017-06-22 19:35:56,710dbda77cade1eced9c735c08e6e38d,5.00,5.00,Assigned Job,594c41ab1e24bf00350a1f11,2017-06-22 18:16:11,vetting,1,vetting,3,11,11,5,5,1,0,0,0
8,18,2017-06-22 19:31:15,c033808b89b50e6f550caf266470f090,4.90,4.90,Assigned Job,594c151273dd9c002873cd98,2017-06-22 15:05:54,sourcing,1,"editing, sourcing, writing",2,10,13,5,5,2,0,1,0
9,20,2017-06-22 19:25:25,710dbda77cade1eced9c735c08e6e38d,5.00,5.00,Assigned Job,594c41ab1e24bf00350a1f11,2017-06-22 18:16:11,vetting,1,"vetting, sourcing",2,12,11,5,5,1,0,0,0


In [47]:
c.execute('''SELECT *
            FROM Assignments
            WHERE Action LIKE  '%Accepted Job';''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data


,index,Event_occurred_at,Analyst,Quality_score_sourcing,Quality_score_writing,Action,Request,Request_created_at,Job,Wait_time_min,Waiting_for,Analysts_available,Analysts_occupied,Total_jobs_available,Review_jobs_available,Vetting_jobs_available,Planning_jobs_available,Editing_jobs_available,Sourcing_jobs_available,Writing_jobs_available
0,0,2017-06-22 19:59:06,9fcbc63ff4c8bea5cea4efad782c87cf,5.00,5.00,Accepted Job,594bec5c95e2ce005840c23a,2017-06-22 12:12:12,review,1,review,0,13,14,4,6,2,0,1,1
1,3,2017-06-22 19:51:01,0e9802516f8a79dd0d45211dd4ee74af,4.50,4.50,Accepted Job,594c1f5cd7e68f0028c9062c,2017-06-22 15:49:48,sourcing,1,"sourcing, writing",1,11,12,5,5,1,0,0,1
2,8,2017-06-22 19:47:03,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Accepted Job,594c1e983b593b00281250ba,2017-06-22 15:46:32,sourcing,4,"sourcing, writing",1,11,11,5,5,1,0,0,0
3,9,2017-06-22 19:47:03,85c7b78e76b5232cd38014ea4cdc8f56,4.35,4.35,Accepted Job,594c1e983b593b00281250ba,2017-06-22 15:46:32,sourcing,4,"sourcing, writing",1,11,11,5,5,1,0,0,0
4,11,2017-06-22 19:44:03,a09c8906073b4c0b75e3100b857b982a,5.00,5.00,Accepted Job,594bec5b95e2ce005840c232,2017-06-22 12:12:11,review,1,"review, planning, editing, sourcing, writing",1,10,11,5,5,1,0,0,0
5,13,2017-06-22 19:40:41,0e9802516f8a79dd0d45211dd4ee74af,4.50,4.50,Accepted Job,594bec83fd2cf400280aa965,2017-06-22 12:12:51,writing,1,"sourcing, writing",2,9,12,6,5,1,0,0,0
6,15,2017-06-22 19:35:59,710dbda77cade1eced9c735c08e6e38d,5.00,5.00,Accepted Job,594c41ab1e24bf00350a1f11,2017-06-22 18:16:11,vetting,1,vetting,2,11,10,5,4,1,0,0,0
7,17,2017-06-22 19:31:18,c033808b89b50e6f550caf266470f090,4.90,4.90,Accepted Job,594c151273dd9c002873cd98,2017-06-22 15:05:54,sourcing,1,"editing, sourcing, writing",2,10,11,5,5,1,0,0,0
8,19,2017-06-22 19:25:28,710dbda77cade1eced9c735c08e6e38d,5.00,5.00,Accepted Job,594c41ab1e24bf00350a1f11,2017-06-22 18:16:11,vetting,1,"vetting, sourcing",1,12,10,5,4,1,0,0,0
9,21,2017-06-22 19:20:23,d8e25a290ea51352bf9100a99c475f6d,5.00,5.00,Accepted Job,594c41482c137900353e0fea,2017-06-22 18:14:32,vetting,1,vetting,1,14,8,3,4,1,0,0,0


In [46]:
c.execute('''SELECT DISTINCT(*) FROM (SELECT *
            FROM Assignments
            WHERE Action LIKE '%Assigned Job%') AS t1
            LEFT JOIN (SELECT *
            FROM Assignments
            WHERE Action LIKE '%Accepted Job%') AS t2
            ON t1.Analyst = t2.Analyst AND t1.Request = t2.Request AND t1.Job = t2.Job;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

OperationalError: near "*": syntax error

In [41]:
# Query the average time it takes for activity to start on a request without the 393 outlier

c.execute('''SELECT t1.Event_occurred_at, t1.Analyst, t2.Event_occurred_at, t2.Analyst, t1.Action, t2.Action, t1.Request, t2.Request, t1.Request_created_at, t2.Request_created_at, t1.Job, t2.Job
             FROM (SELECT *
             FROM Assignments
             WHERE Action LIKE 'Assigned Job') AS t1
             LEFT JOIN (SELECT *
             FROM Assignments WHERE Action LIKE '%Accepted Job%' OR Action LIKE '%Declined Job%') AS t2
             ON t1.Analyst = t2.Analyst AND t1.Request = t2.Request AND t1.Job = t2.Job;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,Event_occurred_at,Analyst,Event_occurred_at,Analyst,Action,Action,Request,Request,Request_created_at,Request_created_at,Job,Job
0,2017-06-22 19:59:02,9fcbc63ff4c8bea5cea4efad782c87cf,2017-06-22 19:59:06,9fcbc63ff4c8bea5cea4efad782c87cf,Assigned Job,Accepted Job,594bec5c95e2ce005840c23a,594bec5c95e2ce005840c23a,2017-06-22 12:12:12,2017-06-22 12:12:12,review,review
1,2017-06-22 19:50:58,85c7b78e76b5232cd38014ea4cdc8f56,2017-06-22 19:51:30,85c7b78e76b5232cd38014ea4cdc8f56,Assigned Job,Declined Job,594bec83fd2cf400280aa965,594bec83fd2cf400280aa965,2017-06-22 12:12:51,2017-06-22 12:12:51,writing,writing
2,2017-06-22 19:50:58,85c7b78e76b5232cd38014ea4cdc8f56,2017-06-22 19:50:58,85c7b78e76b5232cd38014ea4cdc8f56,Assigned Job,Declined Job,594bec83fd2cf400280aa965,594bec83fd2cf400280aa965,2017-06-22 12:12:51,2017-06-22 12:12:51,writing,writing
3,2017-06-22 19:50:58,0e9802516f8a79dd0d45211dd4ee74af,2017-06-22 19:51:01,0e9802516f8a79dd0d45211dd4ee74af,Assigned Job,Accepted Job,594c1f5cd7e68f0028c9062c,594c1f5cd7e68f0028c9062c,2017-06-22 15:49:48,2017-06-22 15:49:48,sourcing,sourcing
4,2017-06-22 19:50:23,85c7b78e76b5232cd38014ea4cdc8f56,2017-06-22 19:51:30,85c7b78e76b5232cd38014ea4cdc8f56,Assigned Job,Declined Job,594bec83fd2cf400280aa965,594bec83fd2cf400280aa965,2017-06-22 12:12:51,2017-06-22 12:12:51,writing,writing
5,2017-06-22 19:50:23,85c7b78e76b5232cd38014ea4cdc8f56,2017-06-22 19:50:58,85c7b78e76b5232cd38014ea4cdc8f56,Assigned Job,Declined Job,594bec83fd2cf400280aa965,594bec83fd2cf400280aa965,2017-06-22 12:12:51,2017-06-22 12:12:51,writing,writing
6,2017-06-22 19:46:56,85c7b78e76b5232cd38014ea4cdc8f56,2017-06-22 19:47:03,85c7b78e76b5232cd38014ea4cdc8f56,Assigned Job,Accepted Job,594c1e983b593b00281250ba,594c1e983b593b00281250ba,2017-06-22 15:46:32,2017-06-22 15:46:32,sourcing,sourcing
7,2017-06-22 19:46:56,85c7b78e76b5232cd38014ea4cdc8f56,2017-06-22 19:47:03,85c7b78e76b5232cd38014ea4cdc8f56,Assigned Job,Accepted Job,594c1e983b593b00281250ba,594c1e983b593b00281250ba,2017-06-22 15:46:32,2017-06-22 15:46:32,sourcing,sourcing
8,2017-06-22 19:44:01,a09c8906073b4c0b75e3100b857b982a,2017-06-22 19:44:03,a09c8906073b4c0b75e3100b857b982a,Assigned Job,Accepted Job,594bec5b95e2ce005840c232,594bec5b95e2ce005840c232,2017-06-22 12:12:11,2017-06-22 12:12:11,review,review
9,2017-06-22 19:40:37,0e9802516f8a79dd0d45211dd4ee74af,2017-06-22 19:40:41,0e9802516f8a79dd0d45211dd4ee74af,Assigned Job,Accepted Job,594bec83fd2cf400280aa965,594bec83fd2cf400280aa965,2017-06-22 12:12:51,2017-06-22 12:12:51,writing,writing


In [ ]:
# Query the time it takes for a request to be complete

c.execute('''SELECT Request, ((strftime('%s', MAX(Event_occurred_at)) - strftime('%s', Request_created_at))/60) AS Total_request_time_mins
            FROM Assignments
            GROUP BY Request
            HAVING Request_created_at <= '2017-06-21 20:15:42'
            ORDER BY Request_created_at;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data.head(15)

In [ ]:
# Query the average time it takes for a request to be complete

c.execute('''SELECT AVG(Total_request_time_mins) AS Average_Total_request_time_mins
            FROM (SELECT Request, ((strftime('%s', MAX(Event_occurred_at)) - strftime('%s', Request_created_at))/60) AS Total_request_time_mins
            FROM Assignments
            GROUP BY Request
            HAVING Request_created_at <= '2017-06-21 20:15:42'
            ORDER BY Request_created_at);''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

There is no indication of when an assignment is complete in this log or if the activity is the last activity. For my best guess at how to calculate this, I only used requests that were created before the events in the assignment logged were log. I used the max Event_occurred_at time as a best estimate to when the request was completed.

## Analysts & Jobs

In [ ]:
# Query for average number of Analysts_available, Analysts_occupied and Total_jobs_available

c.execute('''SELECT AVG(Analysts_available), AVG(Analysts_occupied), AVG(Total_jobs_available)
            FROM Assignments;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

On average there are about 5.38 more jobs available than analysts available, this is an indication that more analysts may be needed.

In [ ]:
# Query for average number of Review_jobs_available, Vetting_jobs_available, Planning_jobs_available, Editing_jobs_available, Sourcing_jobs_available and Writing_jobs_available

c.execute('''SELECT AVG(Review_jobs_available), AVG(Vetting_jobs_available), AVG(Planning_jobs_available), AVG(Editing_jobs_available), AVG(Sourcing_jobs_available), AVG(Writing_jobs_available)
            FROM Assignments;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

In [ ]:
# Query the total number of times each job was being waited on in this log

c.execute('''SELECT (SELECT COUNT(*)
            FROM Assignments
            WHERE Waiting_for LIKE '%review%') AS review_wait_count,
            (SELECT COUNT(*)
            FROM Assignments
            WHERE Waiting_for LIKE '%sourcing%') AS sourcing_wait_count,
            (SELECT COUNT(*)
            FROM Assignments
            WHERE Waiting_for LIKE '%writing%') AS writing_wait_count,
            (SELECT COUNT(*)
            FROM Assignments
            WHERE Waiting_for LIKE '%planning%')AS planing_wait_count,
            (SELECT COUNT(*)
            FROM Assignments
            WHERE Waiting_for LIKE '%editing%') AS editing_wait_count,
            (SELECT COUNT(*)
            FROM Assignments
            WHERE Waiting_for LIKE '%vetting%') AS vetting_wait_count
            FROM Assignments
            LIMIT 1;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

The number of jobs being waited on goes in this order:
 - sourcing
 - writing
 - review
 - editing
 - planning
 - vetting

In [ ]:
# Query the number of times each job is done in the assignment log

c.execute('''SELECT (SELECT COUNT(*)
            FROM Assignments
            WHERE Job LIKE '%review%') AS review_job_count,
            (SELECT COUNT(*)
            FROM Assignments
            WHERE Job LIKE '%sourcing%') AS sourcing_job_count,
            (SELECT COUNT(*)
            FROM Assignments
            WHERE Job LIKE '%writing%') AS writing_job_count,
            (SELECT COUNT(*)
            FROM Assignments
            WHERE job LIKE '%planning%')AS planing_job_count,
            (SELECT COUNT(*)
            FROM Assignments
            WHERE Job LIKE '%editing%') AS editing_job_count,
            (SELECT COUNT(*)
            FROM Assignments
            WHERE Job LIKE '%vetting%') AS vetting_job_count
            FROM Assignments
            LIMIT 1;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

The number of jobs being done on goes in this order:
 - sourcing
 - review
 - writing
 - planning/vetting
 - editing
 
More analysts need to be editing than planning and vetting. More writing should be done than reviewing.

In [ ]:
# Query the number of events for each request

c.execute('''SELECT Request, COUNT(*)
FROM Assignments
GROUP BY Request
HAVING COUNT(*) >2
ORDER BY COUNT(*) DESC;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

There are 56 different requests included in the assignment log. The number of eventsfor each requests ranges from 3 to 43.

In [ ]:
# Query the number of distinct analysts in the assignment log

c.execute('''SELECT DISTINCT Analyst
FROM Assignments
GROUP BY Analyst;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

There are 70 different analysts included inthe assignment log.

In [ ]:
# Query to check if every analysts only has one distinct Quality_score_soucing and one distinct Quality_score_writing

c.execute('''SELECT Analyst, COUNT(DISTINCT Quality_score_sourcing), COUNT(DISTINCT Quality_score_writing)
FROM Assignments
GROUP BY Analyst;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data.head(15)

Since some of the analysts have different values for Quality_Score_sourcing and Quality_score_writing, we know that these values can change.

In [ ]:
# Query the number of times Quality_score_sourcing does not equal Quality_Score_writing

c.execute('''SELECT *
FROM Assignments
WHERE Quality_score_sourcing != Quality_score_writing;''')
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

Since an error was recieved, "ValueError: Length mismatch: Expected axis has 0 elements, new values have 20 elements", no values were returned. This means that every value of Quality_score_sourcing is equivalent to every value of Quality_score_writing.